In [99]:
import requests

from config import settings
from pprint import pprint

In [30]:
def get_news_data(link: str):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0'
    }
    response = requests.get(link,  headers=headers)
    if response.status_code == 200:
        return response.json()
    raise 'Ошибка полученния данных с новостного сайта'

In [31]:
for type_link, links in settings.news_link.items():
    for link in links:
        if type_link == 'mun':
            content = get_news_data(link + '/api/site/matters')
            print(content)
            break
            get_actual_news(content)
        elif type_link == 'reg':
            pass
        else:
            raise f'Неожиданный тип новостных порталов: {type_link}'

{'matters': [{'id': 413020, 'title': '9 апреля 2024: какие сегодня праздники отмечают', 'is_promo': False, 'show_authors': False, 'published_at': '2024-04-09T09:30:00.000+11:00', 'noindex_text': None, 'platform_id': 12, 'lightning': False, 'registration_area': 'bashkortostan', 'is_partners_matter': False, 'is_exclusive': False, 'has_video': False, 'has_audio': False, 'has_gallery': False, 'images_count': 1, 'lead': 'Газета «Южно-Сахалинск сегодня» рассказывает, какие праздники отмечаются в России и мире 9 апреля 2024 года.', 'kind': 'news_item', 'kind_path': '/news', 'image': {'author': None, 'source': '@kues1 / freepik.com', 'description': None, 'width': 800, 'height': 451, 'url': '/attachments/1e463007ba7e84e0adc017974e16d0d720688787/store/crop/0/24/800/451/800/0/0/0/85d204bcb33d68259a8918f7283eaa7da5c8c672a0b9480df766a2585196/85d204bcb33d68259a8918f7283eaa7da5c8c672a0b9480df766a2585196.jpeg.jpg', 'image_90': '/attachments/2b4b9943e2ea16c9fbaef9f489839f5868b9cfa5/store/crop/0/24/800/

### Реализация проверки даты 
путем прибавления к дате публикации +1 день и сравнение с текущим днем

In [54]:
from datetime import datetime, timedelta, timezone
print((datetime.fromisoformat(content['matters'][3]['published_at'])+ timedelta(days=1))> datetime.now(tz=timezone.utc))

False


In [51]:
datetime.now(tz=timezone.utc)

datetime.datetime(2024, 4, 9, 5, 48, 28, 644156, tzinfo=datetime.timezone.utc)

### Запись полученных данных в pydantic модель

In [79]:
import models.pydantic_mun_model


importlib.reload(models.pydantic_mun_model)

from models.pydantic_mun_model import News

new_dict = {}

In [80]:
new_dict['path'] = content['matters'][0]['path']
new_dict['title']  = content['matters'][0]['title']
new_dict['published_at'] = content['matters'][0]['published_at']
new_dict['images_count'] = content['matters'][0]['images_count']
new_dict['lead'] = content['matters'][0]['lead']

new_news = News(**new_dict)


{'path': '/news/society/2024-04-09/9-aprelya-2024-kakie-segodnya-prazdniki-otmechayut-413020', 'title': '9 апреля 2024: какие сегодня праздники отмечают', 'published_at': '2024-04-09T09:30:00.000+11:00', 'images_count': 1, 'lead': 'Газета «Южно-Сахалинск сегодня» рассказывает, какие праздники отмечаются в России и мире 9 апреля 2024 года.'}


In [100]:
# Создание объекта без выборки словаря
cur_news = News(**content['matters'][0])

### Получение статьи

In [95]:
# получение ссылки
article_link = link +'api/site/matters'+ new_news.path
# получение данных с сылки
article_data = requests.get(article_link)

if article_data.status_code == 200:
    article_json = article_data.json()
else:
    print(article_data)

https://sakhizdat.ru/api/site/matters/news/society/2024-04-09/9-aprelya-2024-kakie-segodnya-prazdniki-otmechayut-413020


In [117]:
importlib.reload(models.pydantic_mun_model)

from models.pydantic_mun_model import Article

cur_article = Article(**article_json)

In [104]:
cur_news.article = cur_article
print(cur_news)

title='9 апреля 2024: какие сегодня праздники отмечают' path='/news/society/2024-04-09/9-aprelya-2024-kakie-segodnya-prazdniki-otmechayut-413020' published_at=datetime.datetime(2024, 4, 9, 9, 30, tzinfo=TzInfo(+11:00)) images_count=1 lead='Газета «Южно-Сахалинск сегодня» рассказывает, какие праздники отмечаются в России и мире 9 апреля 2024 года.' article=Article(title='9 апреля 2024: какие сегодня праздники отмечают', published_at=datetime.datetime(2024, 4, 9, 9, 30, tzinfo=TzInfo(+11:00)), lead='Газета «Южно-Сахалинск сегодня» рассказывает, какие праздники отмечаются в России и мире 9 апреля 2024 года.', tag=None, content=None)


In [110]:
response = requests.get('https://sakhizdat.ru/api/site/matters/articles/persona-1/2024-04-08/viktoriya-matvienko-vospitatel-dolzhen-nauchit-detey-obschatsya-druzhit-i-proschat-412754')
rez = response.json()
rez

Article(title='Виктория Матвиенко: воспитатель должен научить детей общаться, дружить и прощать', published_at=datetime.datetime(2024, 4, 8, 10, 28, tzinfo=TzInfo(+11:00)), lead='Жюри муниципального этапа конкурса «Педагог года - 2024» , который традиционно прошел в Южно-Сахалинске,  отметило ее в специальной номинации «За сохранение культурных традиций». ', tag=None, content=None, image=Image(author=None, source=None, description=None, width=631, height=356, image_90='/attachments/96955f96510ed9619513f54983f64f493af79eb6/store/crop/84/0/631/356/90/0/0/0/2c98659daa0974b9334c652fcbcea445cd0649c311232bd9a9cb130714ee/2c98659daa0974b9334c652fcbcea445cd0649c311232bd9a9cb130714ee.jpeg.jpg', image_250='/attachments/084aca05b52a0bcf97a22637ad9bb8ef7caeba74/store/crop/84/0/631/356/250/0/0/0/2c98659daa0974b9334c652fcbcea445cd0649c311232bd9a9cb130714ee/2c98659daa0974b9334c652fcbcea445cd0649c311232bd9a9cb130714ee.jpeg.jpg', image_800='/attachments/20a2fd873f1675ee41d35bce025f35adf9b58d79/store/cro

In [118]:
rez_article = Article(**rez)
rez_article

Article(title='Виктория Матвиенко: воспитатель должен научить детей общаться, дружить и прощать', published_at=datetime.datetime(2024, 4, 8, 10, 28, tzinfo=TzInfo(+11:00)), lead='Жюри муниципального этапа конкурса «Педагог года - 2024» , который традиционно прошел в Южно-Сахалинске,  отметило ее в специальной номинации «За сохранение культурных традиций». ', tags=[Tag(title='педагог', slug='pedagog', path='/tags/pedagog'), Tag(title='конкурс', slug='konkurs', path='/tags/konkurs'), Tag(title='детский сад', slug='detskiy-sad', path='/tags/detskiy-sad'), Tag(title='семья', slug='semya', path='/tags/semya'), Tag(title='образование', slug='obrazovanie', path='/tags/obrazovanie'), Tag(title='традиции', slug='traditsii', path='/tags/traditsii')], content_blocks=[Content(position=0, kind='common', text='<p><em>Общий стаж работы в сфере образования - 13 лет. За это время прошла профессиональную переподготовку и освоила методы, которые помогают увлечь полезным даже самых непоседливых. В 2018-м 

In [119]:
rez_article.tags

[Tag(title='педагог', slug='pedagog', path='/tags/pedagog'),
 Tag(title='конкурс', slug='konkurs', path='/tags/konkurs'),
 Tag(title='детский сад', slug='detskiy-sad', path='/tags/detskiy-sad'),
 Tag(title='семья', slug='semya', path='/tags/semya'),
 Tag(title='образование', slug='obrazovanie', path='/tags/obrazovanie'),
 Tag(title='традиции', slug='traditsii', path='/tags/traditsii')]

In [1]:
import requests
from mun_model import News, Article
link = 'https://sakhizdat.ru/'
# Получаем новости
response_news = requests.get('https://sakhizdat.ru/api/site/matters')
for news in response_news.json()['matters']:
    cur_news = News(**news)
    # Получаем ссылку на статью
    article_link = 'https://sakhizdat.ru/api/site/matters' + cur_news.path
    # Получаем статью
    article_response = requests.get(article_link).json()
    article = Article(**article_response)
    article.site_link = link
    article.image.image_90 = (link +
                                article.
                                image.
                                image_90[1:])
    article.image.image_800 = (link +
                                article.
                                image.
                                image_800[1:])
    article.image.image_250 = (link +
                                article.
                                image.
                                image_250[1:])
    article.image.image_1600 = (link +
                                article.
                                image.
                                image_1600[1:])
    break

Image(author=None, source='loon.site', description=None, width=794, height=449, image_90='https://sakhizdat.ru/attachments/a1f317a6fe142b900686ff0115f3e53d8a075da7/store/crop/0/0/794/449/90/0/0/0/2b98ea6fa5c030f41d7357138c29f5aa40aab423dd6db0b7bee7a7860204/2b98ea6fa5c030f41d7357138c29f5aa40aab423dd6db0b7bee7a7860204.jpeg.jpg', image_250='https://sakhizdat.ru/attachments/a6208d394895b0d09011297c47d1fc5631f4728a/store/crop/0/0/794/449/250/0/0/0/2b98ea6fa5c030f41d7357138c29f5aa40aab423dd6db0b7bee7a7860204/2b98ea6fa5c030f41d7357138c29f5aa40aab423dd6db0b7bee7a7860204.jpeg.jpg', image_800='https://sakhizdat.ru/attachments/8b7b6e29f7417667c6dbe8d4337648385e0302c0/store/crop/0/0/794/449/800/0/0/0/2b98ea6fa5c030f41d7357138c29f5aa40aab423dd6db0b7bee7a7860204/2b98ea6fa5c030f41d7357138c29f5aa40aab423dd6db0b7bee7a7860204.jpeg.jpg', image_1600='https://sakhizdat.ru/attachments/c185b238f52e066f66d1019ae5d796eb3229a611/store/crop/0/0/794/449/1600/0/0/0/2b98ea6fa5c030f41d7357138c29f5aa40aab423dd6db0b7b

In [23]:
SOURCE_LIST = [
    'Южно-Сахалинск - Газета «Южно-Сахалинск сегодня» - https://sakhizdat.ru/',
    'Корсаковский ГО - Газета «Восход» - https://восход65.рф/',
    'Долинский ГО - Газета «Долинская правда» - https://dolinsk.today/',
    'Ногликский ГО - Газета «Знамя труда» - https://noglgazeta.ru/',
    'Александровск-Сахалинский - Газета «Красное знамя» - https://znamya65.ru/',
    'Курильский - Газета «Красный маяк» - https://iturup.news/',
    'Северо-Курильскиий - Газета «Курильский рыбак» — https://krsevkur.ru/',
    'Южно-Курильский - Газета «На рубеже» — https://kurilnews.ru/',
    'Невельский - Газета «Невельские новости» — https://nevnews.info/',
    'Макаровий - Газета «Новая газета» — https://gazetamakarov.ru/',
    'Смирныховский - Газета «Новая жизнь» — https://alsakh.ru/',
    'Тымовский - Газета «Тымовский вестник» — https://tymnews.ru/',
    'Анивский - Газета «Утро Родины» - https://aniva-utro.ru/',
    'Холмский - Газета «Холмская панорама» - https://kholmsk.info/',
    'Попронайский - Газета «Экспресс» - https://express65.ru/',
    'Томаринский - Газета «Вести Томари» - https://vesti-tomari.ru/',
]

for source in SOURCE_LIST:
    print(source)


Южно-Сахалинск - Газета «Южно-Сахалинск сегодня» - https://sakhizdat.ru/
Корсаковский ГО - Газета «Восход» - https://восход65.рф/
Долинский ГО - Газета «Долинская правда» - https://dolinsk.today/
Ногликский ГО - Газета «Знамя труда» - https://noglgazeta.ru/
Александровск-Сахалинский - Газета «Красное знамя» - https://znamya65.ru/
Курильский - Газета «Красный маяк» - https://iturup.news/
Северо-Курильскиий - Газета «Курильский рыбак» — https://krsevkur.ru/
Южно-Курильский - Газета «На рубеже» — https://kurilnews.ru/
Невельский - Газета «Невельские новости» — https://nevnews.info/
Макаровий - Газета «Новая газета» — https://gazetamakarov.ru/
Смирныховский - Газета «Новая жизнь» — https://alsakh.ru/
Тымовский - Газета «Тымовский вестник» — https://tymnews.ru/
Анивский - Газета «Утро Родины» - https://aniva-utro.ru/
Холмский - Газета «Холмская панорама» - https://kholmsk.info/
Попронайский - Газета «Экспресс» - https://express65.ru/
Томаринский - Газета «Вести Томари» - https://vesti-tomar

In [2]:
# если необходимо сделать сплит по нескольким элементам
# можно реплейсом все привести к единому виду
source.replace('—', '-').split(' - ')

NameError: name 'source' is not defined

In [1]:
import importlib
import config
import sqlalchemy_model


importlib.reload(config)
importlib.reload(sqlalchemy_model)


from config import settings
from sqlalchemy_model import SourceOrm

for source in settings.news_link:
    r = SourceOrm(**source)
    print(r)
    break


### ASTV парсер

In [13]:
import logging
from datetime import datetime, timedelta


logging.basicConfig(
    format="#{levelname:8} {name}:{funcName} - {message}",
    style='{'
)
log = logging.getLogger(__name__)
log.setLevel('DEBUG')


import requests
from bs4 import BeautifulSoup as bs
# доработать изменение цифры до тех пор пока не будут позавчерашине новости
response = requests.get('https://astv.ru/news/main/fresh/2')


In [14]:
soup = bs(response.text, 'lxml')
soup.text

'\n\nНовости Сахалина и области – лента новостей - страница 2 - astv.ru\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nновости\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nВход\nРегистрация\n\n\n\n\n\nНовости\n\n\n\n\n\n\nМир\nОбщество\nПолитика и экономика\nПроисшествия\nКонфликт\nКультура\nСпорт\nНаш день\nГороскоп\nРазгрузка\n\n\nФото\nВидео\nСюжеты\n\n\n\n\n\n\n\n\n\n\nВидео массовой драки с угрозами лопатой и розочкой распространяется в сахалинских соцсетях\n\n\n\n\n\n\n\n\n\n\n\nСахалинские спасатели обнаружили лодку и вещи пропавшего на Тунайче мужчины\n\n\n\n\n\n\n\n\n\n\n\n"Их тут несколько сотен": мёртвые вороны пугают южносахалинцев на кладбище\n\n\n\n\n\nОбсуждают\n\n"Идём убивать": школьники на Сахалине устроили травлю двух девочек и выложили видео с нацистской символикой\nПродажу алкоголя в России хотят ограничить до 7 часов вечера\nМужчина на Сахалине волоком д

In [15]:
# получение блоков всех новостей на этой странице
news = soup.find_all('div', class_='item width-100-tiny')

### IMAGE

In [16]:
def get_image_dict(img_bs):
    return {
        'source': img_bs.get('src'),
        'image_90': img_bs.get('src'),
        'image_250': img_bs.get('src'),
        'image_800': img_bs.get('src'),
        'image_1600': img_bs.get('src'),
        'width': img_bs.get('width'),
        'height': img_bs.get('height'),
        }

# получение фотографии
img_bs = news[0].find_all('img')[0]
print(img_bs.get('src'))
print(img_bs.get('width'))
print(img_bs.get('height'))
base_url = 'https://astv.ru'
article_url = news[0].find('a').get('href')

https://astv.ru/content/NewsImage/98/bc/98bc67fb-a37a-49b1-86fd-695dc114c763_5.jpg
288
178


In [17]:
# функция которая преобразует строку фомата "вчера, в 18:18,"
# в datetime
def convert_str_to_date(string_time: str) -> datetime:
    log.debug(string_time)
    if 'вчера' in string_time:
        current_datetime = datetime.now()- timedelta(days=1)
    elif 'сегодня':
        current_datetime = datetime.now()
    hours, minutes = map(int, string_time[:-1].split()[-1].split(':'))
    published_at = datetime(
        current_datetime.year,
        current_datetime.month,
        current_datetime.day,
        hours,
        minutes,
        0
        )
    return published_at


title = news[0].find_all('a')[-1].text
log.info(f'{title=}')
# получение даты публикации
date_string = news[0].select_one('span.ico-p:not([class*=" "]):not([title])').text
log.debug(f'Данные о дате публикации: {date_string}')
published_at = convert_str_to_date(date_string)
log.info(f'{published_at=}')
rubric_title = news[0].find_all('a')[-2].text
# альтернативный вариант получения
# rubric_title = news[0].find('img').get('alt')
log.info(f'{rubric_title=}')

#INFO     __main__:<module> - title='Команду "Сахалинские Акулы" покинула часть игроков'
#DEBUG    __main__:<module> - Данные о дате публикации: вчера, в 17:48,
#DEBUG    __main__:convert_str_to_date - вчера, в 17:48,
#INFO     __main__:<module> - published_at=datetime.datetime(2024, 5, 1, 17, 48)
#INFO     __main__:<module> - rubric_title='Спорт'


In [18]:
article_response = requests.get(base_url + article_url)
# article_response = requests.get('https://astv.ru/news/society/2024-04-24-blagoustrojstvo-na-minimalkah-v-holmske-pokrasili-slomannye-lavochki')
article_soup = bs(article_response.text, 'lxml')

In [21]:
article_response = requests.get('https://astv.ru/news/criminal/2024-04-24-ochevidcev-stolknoveniya-krossovera-i-minivena-ishut-v-yuzhno-sahalinske')
# article_response = requests.get('https://astv.ru/news/society/2024-04-24-blagoustrojstvo-na-minimalkah-v-holmske-pokrasili-slomannye-lavochki')
article_soup = bs(article_response.text, 'lxml')

### TAGS

#### ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [22]:
tags_list = []
for tags_block in article_soup.find_all('div', 'block-tegs-text'):
    for tags in tags_block.find_all('a'):
        tags_list.append(tags.text)
# tags_list = [elem.text for elem in article_soup.find('div', 'block-tegs-text').find_all('a')]
log.info(tags_list)

#INFO     __main__:<module> - ['поиск', 'розыск', 'поиск очевидцев', 'дтп', 'авария', 'Южно-Сахалинск']


#### ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [52]:
log.setLevel('DEBUG')
content_page = article_soup.find('div', class_='content newsDetails')
elem_list = content_page.find('div', attrs={'id': 'mainContentFromPage'})
lead = content_page.find('div', class_='h3 lid').text
log.info(f'{lead=}')
author = article_soup.find('span', attrs={"itemprop":"author"}).text.strip().split()[-1]
log.info(f'{author=}')

elem_list_p = elem_list.find_all('p')
for elem in elem_list_p:
    if elem.find('img'):
        log.debug('обработка фотки')
    elif elem.find('iframe'):
        log.debug('скип видео')
    else:
        log.debug(f'вывод{elem.prettify}')

elem_list_div = elem_list.find_all('div')
img_dict = {}

for index, elem in enumerate(elem_list_div):
    if elem.find('a'):
        img_gallery_bs = elem.find('a')
        img_link_bs = img_gallery_bs.find('img')
        if not img_link_bs:
            continue
        img_short_link = img_link_bs.get('src')
        log.debug(img_short_link)
        img_full_link = 'https://astv.ru' + img_short_link
        if not img_dict.get(img_full_link):
            img_dict[img_full_link] = {
                    'image_90': img_full_link,
                    'image_250': img_full_link,
                    'image_800': img_full_link,
                    'image_1600': img_full_link,
                    'width': img_link_bs.get('width'),
                    'height': img_link_bs.get('height'),
                }
    else:
        if elem.text:
            log.debug(elem.prettify)
log.debug(img_dict.values())
# for img_gallery_link in img_gallery_link_set:
#     response = requests.get(img_gallery_link)
#     if response.status_code == 200:
#         img_gallery_bs = bs(response.text, 'lxml')
#         log.info(img_gallery_bs)
#         img_link_list = img_gallery_bs.find_all('div', class_='gallery_main_image_a')
#         log.info(img_link_list)
#     else:
#         log.error(f'Запрос по ссылке {img_gallery_link} не дал результатов\n'
#                   f'{response.status_code=}')


#INFO     __main__:<module> - lead='Если вы застали момент ДТП или он попал на ваш видеорегистратор, откликнитесь'
#INFO     __main__:<module> - author='АСТВ'
#DEBUG    __main__:<module> - <bound method Tag.prettify of <div>
	Госавтоинспекция Южно-Сахалинска ищет очевидцев аварии, что произошла на перекрёстке в областном центре. Столкнулись кроссовер и минивэн. Обошлось без травм. </div>>
#DEBUG    __main__:<module> - <bound method Tag.prettify of <div>
	 </div>>
#DEBUG    __main__:<module> - https://astv.ru/content/Photogallery/e4/ce/e4ce4f9c-8610-4f1f-9f55-6a45c2f490c2_s.jpg
#DEBUG    __main__:<module> - https://astv.ru/content/Photogallery/e4/ce/e4ce4f9c-8610-4f1f-9f55-6a45c2f490c2_s.jpg
#DEBUG    __main__:<module> - https://astv.ru/content/Photogallery/e4/ce/e4ce4f9c-8610-4f1f-9f55-6a45c2f490c2_s.jpg
#DEBUG    __main__:<module> - <bound method Tag.prettify of <div class="text-line"><span>Галерея</span></div>>
#DEBUG    __main__:<module> - https://astv.ru/content/Photogallery/e4/ce/

In [39]:
print(response.text)

<!doctype html>
<html lang="ru">
<head>
	<title>Очевидцев столкновения кроссовера и минивэна ищут в Южно-Сахалинске - Новости Сахалинской области. Происшествия – Фотогалерея, фото 1 - astv.ru</title>
		<meta name="keywords" content="поиск, розыск, поиск очевидцев, дтп, авария, Южно-Сахалинск" />
	<meta name="description" content="Если вы застали момент ДТП или он попал на ваш видеорегистратор, откликнитесь" />

	<link rel="manifest" href="/manifest.json" />

	<meta name="viewport" content="initial-scale=1, maximum-scale=1" />

	<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
	<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon" />
	<!-- html5.js for IE less than 9 -->
	<!--[if lt IE 9]>
		<script src="https://css3-mediaqueries-js.googlecode.com/svn/trunk/css3-mediaqueries.js"></script>
	<![endif]-->
	<!--[if lt IE 9]>
		<script src="https://html5shim.googlecode.com/svn/trunk/html5.js"></script>
	<![endif]-->

	<link rel="alternate" type="application/r